In [1]:
import pandas as pd
import numpy as np
import customtkinter as ctk

In [ ]:
'''
VALUES TO ASK ABOUT
in flow boundary conditions, the only if bc=62 - is this handled by another file, or do I need to handle it, what is the 0.03
On the first line of free surface parameters, what parameter is the 0.65
On time series, what is all the values for point #1
Why are gx and gy and gz defined twice, once for numeric parameters and once for free surface parameters - how to handle

#TO DO: Change 0s so they are not strings, or that is handled effectively
#TODO: Ask about outflow and neumannNBC, they are both values of 2, is that fine? Confirm again on boundary conditions choices)
#TODO: Add "NONE" option for some dropdowns
'''

In [ ]:
'''
#TODO: Repeat fractions n times for user input in LPT file


handle submit button: dictionary, df and file creation

Add calculate button to handle wrong input for mesh size, ensure cells per block are not more than 100

Error handling for incorrect entries/missing entries


3 more pages >.<

organize code and make it pretty
'''

In [ ]:
'''
NEXT STEPS for handling submit
Change the way the dictionary is processed so it handles the questions effectively - for both advanced and normal
Ensure defaults are being managed in advanced
Access keys from dictionary and store user input next to keys
Process user input through value mapping
Call df creation
Call file creation
'''

In [18]:
# Initialize the CustomTkinter app
ctk.set_appearance_mode("dark")  # Options: "light", "dark", "system"
ctk.set_default_color_theme("green")  # Options: "blue", "green", "dark-blue"

root = ctk.CTk()
root.title("Automated File Creation")
root.geometry("500x400")

# Create a scrollable frame
scrollable_frame = ctk.CTkScrollableFrame(root, width=480, height=360)
scrollable_frame.pack(pady=10, padx=10, fill="both", expand=True)

# Add input fields to the scrollable frame
ctk.CTkLabel(scrollable_frame, text="Step 1: Control File", font=("Arial", 16)).pack(pady=10,anchor="w")

# Function to open a popup with advanced options
def open_advanced_popup(sections):
    # Create a new window (popup) for advanced options
    popup_window = ctk.CTkToplevel()
    popup_window.title("Advanced Options")
    popup_window.geometry("500x400")

    # Create a scrollable frame for the popup window (for advanced sections)
    popup_scrollable_frame = ctk.CTkScrollableFrame(popup_window,width=480,height=360)
    popup_scrollable_frame.pack(fill="both", expand=True)

    for section in advanced_sections:
        create_section(section, popup_scrollable_frame)

# Function to toggle section visibility
def toggle_section(section):
    frame = section["frame"]
    if frame.winfo_ismapped():
        frame.pack_forget()  # Hide the frame
    else:
        frame.pack(fill="x")  # Show the frame
        text_color="blue",
        width=0,  # Shrink to fit text
        anchor="w"  # Align to the left
    toggle_button.pack(anchor="w", pady=10, padx=10)

    # Dictionary to store frames for each section
    for section in advanced_sections:
        section_frame = create_section_frame(scrollable_frame, section)
        section["frame"] = section_frame  # Attach the frame to the section for toggling
        section_frame.pack_forget()  # Hide the section initially

def create_section_frame(parent, section):
    # Create a frame for the section
    section_frame = ctk.CTkFrame(parent)

    # Add static header
    section_header = ctk.CTkLabel(section_frame, text=section["section_name"], font=("Arial", 14, "bold"))
    section_header.pack(fill="x", padx=10, pady=5)

    # Add inputs for all questions in the section
    for question in section["questions"]:
        add_input_widget(section_frame, *question)

    return section_frame

def add_input_widget(parent_frame, question, input_type, *options):
    # Add label for the question
    label = ctk.CTkLabel(parent_frame, text=question)
    label.pack(anchor="w", padx=10)

    # Add appropriate input widget
    if input_type in ["float", "int"]:
        entry = ctk.CTkEntry(parent_frame, placeholder_text=f"Enter {input_type}")
        entry.pack(fill="x", padx=10, pady=2)
    elif input_type == "dropdown":
        dropdown = ctk.CTkOptionMenu(parent_frame, values=options[0])
        dropdown.pack(fill="x", padx=10, pady=2)
    elif input_type == "radio":
        radio_frame = ctk.CTkFrame(parent_frame)
        radio_frame.pack(fill="x", padx=10, pady=2)
        for option in options[0]:
            radio_button = ctk.CTkRadioButton(radio_frame, text=option, variable=ctk.StringVar(), value=option)
            radio_button.pack(anchor="w")

# List of sections, each containing questions
sections = [
    {
        "section_name": "Numeric Parameters",
        "questions": [
            {"key":"keyword","question": "What kind of scenario do you want to simulate?", 
             "input_type": "dropdown","options": ["channel", "cavity", "column"]},
            {"key":"ubulk","question":"What is the ubulk value?","input_type": "float"},
            {"key":"mesh_size","question":"What is the mesh size?","input_type": "float"},
            {"key":"fluid","question":"What will be the main fluid, water or air?",
             "input_type": "radio","options": ["water", "air"]},
            {"key":"differencing","question":"What discretisation scheme do you choose?",
             "input_type": "dropdown","options":["2nd order CDS", "4th order CDS","WENO"]},
            {"key":"dt","question":"Which timestep do you want?","input_type":"float"},
            {"key":"itime_end","question":"How many timesteps do you want to run?","input_type":"int"},
            {"key":"restart","question": "Is this a restart?","input_type":"radio","options":["yes","no"]},
            {"key":"reinitmean","question":"Do you want to calculate statistics from scratch?",
             "input_type":"radio","options":["YES","NO"]},
            {"key":"n_out","question":"How often do you want to produce outputs (tecbins)?","input_type":"int"},
            {"key":"l_transient","question":"Do you want to produce transient outputs?",
             "input_type":"radio","options":["yes","no"]},
            {"key":"results","question":"How often do you want to produce transient outputs? (If none, enter 0)",
             "input_type":"int"}
        ]
    },
    {
        "section_name": "Flow Boundary Conditions",
        "questions": [
            {"key":"west_boundary_condition","question":"Which west boundary condition option do you want?","input_type":
                                 "dropdown","options": ["inflow","slip", "no-slip", 
                                "smooth log law","1/7th power law", "prescribed inflow", "SEM", 
                                "periodic condition", "rough log law", "1/6th law", "1/7th law","1/8th law"]},
            {"key":"east_boundary_condition","question":"Which east boundary condition option do you want?", 
                                 "input_type":"dropdown","options": ["outflow", "convective", "periodic", "neumann",
                                "slip", "no-slip","smooth log law", "rough log law", "1/6th law", "1/7th law", "1/8th law"]},
            {"key":"south_boundary_condition","question":"Which south boundary condition option do you want?",
                                 "input_type": "dropdown","options": ["slip", "no-slip", "periodic", "smooth log law", 
                                "rough log law", "1/6th law", "1/7th law", "1/8th law"]},
            {"key":"north_boundary_condition","question":"Which north boundary condition option do you want?", 
                                 "input_type":"dropdown","options": ["slip", "no-slip", "periodic", "smooth log law", 
                                "rough log law", "1/6th law", "1/7th law", "1/8th law"]},
            {"key":"bottom_boundary_condition","question":"Which bottom boundary condition option do you want?",
                                 "input_type": "dropdown","options": ["slip", "no-slip", "periodic", "smooth log law", 
                                "rough log law", "1/6th law", "1/7th law", "1/8th law"]},
            {"key":"top_boundary_condition","question":"Which top boundary condition option do you want?", 
                                 "input_type":"dropdown","options": ["slip", "no-slip", "periodic", "smooth log law", 
                                "rough log law", "1/6th law", "1/7th law", "1/8th law"]}
        ]
    },
    {
        "section_name": "Modelling Options",
        "questions": [
            {"key":"time_averaging","question":"At which ctime do you want to start collecting statistics?",
             "input_type": "float"}, 
            {"key":"SGS_model","question":"Do you want an SGS model?","input_type": "radio","options": ["yes", "no"]},
            {"key":"SGS_model_choice","question":"Which SGS model do you want?", "input_type":"dropdown",
             "options": ["None","Smagorinsky","WALE","OneEqnModel","k-eps model (RANS)"]},
            {"key":"limb","question":"Do you want immersed boundaries to be on or off?","input_type": "radio",
             "options": ["on", "off"]},
            {"key":"lenergy","question":"Do you want energy equation?","input_type":"radio","options": ["yes", "no"]},
            {"key":"lpt","question":"Do you want to create lagrangian particles?","input_type":"radio",
             "options": ["yes", "no"]},
            {"key":"lscalar","question":"Passive scalar?","input_type":"radio","options": ["yes", "no"]},
            {"key":"l_active_scalar","question":"Active scalar?","input_type":"radio","options":["yes","no"]}
        ]
    }
]

advanced_sections = [
    {
        "section_name": "Numeric Parameters",
        "questions": [
            {"key":"pr","question":"pr:","input_type":"float","default":0.72},
            {"key":"turb_schmidt","question":"turb_schmidt:","input_type":"float","default":0.6},
            {"key":"beta","question":"beta:","input_type": "float","default":0.0034},
            {"key":"gx","question":"gx:","input_type":"float","default":0.0},
            {"key":"gy","question":"gy:", "input_type":"float","default":0.0},
            {"key":"gz","question":"gz:", "input_type":"float","default":-9.81},
            {"key":"convection_scheme","question":"convection scheme:","input_type": "dropdown",
             "options":["Exp.Euler","AdamsBashfort","RK2","RK3"],"default":"RK2"},
            {"key":"diffusion_scheme","question":"diffusion scheme:", "input_type":"dropdown",
             "options":["Imp.Euler","CrankNicholson","ExplicitRungeKuttaDiff"],"default":"ExplicitRungeKuttaDiff"},
            {"key":"solver","question":"solver:", "input_type":"dropdown","options":["sip","mg"],"default":"mg"},
            {"key":"multigrid_step","question":"multigrid step (ngrid):","input_type": "int","default":1},
            {"key":"multigrid_iteration_scheme","question":"multigrid iteration scheme:", "input_type":"dropdown",
             "options":["GSS","TDMA","RBGSS"],"default":"TDMA"},
            {"key":"multigrid_max_iter","question":"multigrid maximum iteration per timestep:","input_type": "int",
             "default":30},
            {"key":"restriction_iter","question":"restriction iter:","input_type":"int","default":2},
            {"key":"prolongation_iter","question":"prolongation iter:", "input_type":"int","default":1},
            {"key":"variable_fixed","question":"variable or fixed dt:","input_type": "radio","options":["variable","fixed"],
            "default":"fixed"},
            {"key":"sweeps","question":"sweeps:","input_type": "int","default":25},
            {"key":"safety_factor","question":"safety factor:","input_type": "float","default":0.2},
            {"key":"niter","question":"niter:", "input_type":"int","default":20},
            {"key":"eps","question":"eps:","input_type": "float","default":1.0E-5},
            {"key":"nswp_1","question":"nswp(1):","input_type": "int","default":5},
            {"key":"nswp_2","question":"nswp(2):", "input_type":"int","default":5},
            {"key":"nswp_3","question":"nswp(3):", "input_type":"int","default":5},
            {"key":"nswp_4","question":"nswp(4):","input_type": "int","default":20}
        ]
    },
    {
        "section_name": "Flow Boundary Conditions",
        "questions": [
            {"key":"friction_coeff","question":"Friction coefficient:","input_type":"radio",
             "options": ["manning", "equivalent sand"],"default":"manning"},
            {"key":"friction_coeff_value","question":"What is the value of the friction coefficient?", "input_type":"float",
            "default":0.03},
            {"key":"save_inflow_data","question":"Save inflow data (precursor sim.):","input_type":"radio",
             "options":["True","False"],"default":"False"},
            {"key":"inlet_num","question":"Number of inlets:","input_type":"int","default":5000}
        ]
    },
    {
        "section_name": "Synthetic Eddy Method",
        "questions": [
            {"key":"velocity_profile","question":"Velocity Profile:","input_type":"radio",
             "options": ["Uniform", "1/7th PL"],"default":"1/7th PL"},
            {"key":"turbulence_intensity","question":"Turbulence intensity:","input_type":"float","default":0.1},
            {"key":"inlet_profile_num","question":"Number inlet profiles:","input_type":"int","default":1000}
        ]
    },
    {
        "section_name": "Modelling Options",
        "questions": [
            {"key":"time_averaging","question":"time_averaging:","input_type":"radio","options": ["True", "False"],
             "default":"True"},
            {"key":"noise","question":"noise:","input_type":"float","default":0.0},
            {"key":"LMR","question":"LMR:","input_type":"radio",
             "options":["old ghost cell approach","new ghost cell approach"],"default":"new ghost cell approach"},
            {"key":"normal_ghost_velcoity_interpolation","question":"normal_ghost_velocity_interpolation",
             "input_type":"radio","options": ["2nd order", "4th order"],"default":"4th order"},
            {"key":"lrough","question":"lrough:","input_type":"radio","options":["True","False"],"default":"False"},
            {"key":"openmp_threads","question":"OpenMP threads:","input_type":"int","default":1},
            {"key":"pl_ex","question":"pl_ex (# of extra ghost planes. pl_ex=0 -> blocks have only 1 ghost layer):", 
             "input_type":"int","default":2},
            {"key":"th","question":"Th:","input_type":"int","default":293},
            {"key":"tc","question":"Tc:","input_type":"int","default":293},
        ]
    },
    {
        "section_name": "Energy Boundary Conditions",
        "questions": [
            {"key":"west_temp_boundary","question":"West Temp Boundary Conditions:","input_type":"dropdown",
             "options": ["Periodic", "Adiabatic","Cold Surface","Hot Surface"],"default":"Adiabetic"},
            {"key":"east_temp_boundary","question":"East Temp Boundary Conditions:","input_type":"dropdown",
             "options": ["Periodic", "Adiabatic","Cold Surface","Hot Surface"],"default":"Adiabetic"},
            {"key":"south_temp_boundary","question":"South Temp Boundary Conditions:","input_type":"dropdown",
             "options": ["Periodic", "Adiabatic","Cold Surface","Hot Surface"],"default":"Adiabetic"},
            {"key":"north_temp_boundary","question":"North Temp Boundary Conditions:","input_type":"dropdown",
             "options": ["Periodic", "Adiabatic","Cold Surface","Hot Surface"],"default":"Adiabetic"},
            {"key":"bottom_temp_boundary","question":"Bottom Temp Boundary Conditions:","input_type":"dropdown",
             "options": ["Periodic", "Adiabatic","Cold Surface","Hot Surface"],"default":"Cold Surface"},
            {"key":"top_temp_boundary","question":"Top Temp Boundary Conditions:","input_type":"dropdown",
             "options": ["Periodic", "Adiabatic","Cold Surface","Hot Surface"],"default":"Hot Surface"}
        ]
    },
    {
        "section_name": "Free Surface Parameters",
        "questions": [
            {"key":"l_lsm","question":"L_LSM:","input_type":"radio","options": ["True", "False"],"default":"False"},
            {"key":"reinit","question":"reinit:","input_type":"radio","options": ["True","False"],"default":"False"},
            {"key":"ntime_reinit","question":"ntime_reinit:","input_type":"int","default":3},
            {"key":"reldif_LSM","question":"reldif_LSM:","input_type":"float","default":0.001},
            {"key":"length","question":"length:","input_type":"float","default":0.02},
            {"key":"accuracy","question":"accuracy:","input_type":"radio","options":["HJ_ENO","HJ_WENO"],"default":"HJ_WENO"},
            {"key":"missing_value","question":"missing value:","input_type":"float","default":0.65},
            {"key":"lends","question":"LENDS:","input_type":"radio","options":["True","False"],"default":"False"},
            {"key":"L_LSMbase","question":"L_LSMbase:","input_type":"radio","options":["True","False"],"default":"False"},
            {"key":"L_LSMinit","question":"L_LSMinit:","input_type":"radio","options":["True","False"],"default":"False"},
            {"key":"L_anim_phi","question":"L_anim_phi:","input_type":"radio","options":["True","False"],"default":"False"},
            {"key":"L_anim_grd","question":"L_anim_grd:","input_type":"radio","options":["True","False"],"default":"False"},
            {"key":"densl","question":"densl:","input_type":"float","default":1000.0},
            {"key":"densg","question":"densg:","input_type":"float","default":1.205},
            {"key":"nul","question":"nul:","input_type":"float","default":1.0E-6},
            {"key":"nug","question":"nug:","input_type":"float","default":1.5E-5},
            {"key":"gx","question":"gx:","input_type":"float","default":0.0},
            {"key":"gy","question":"gy:","input_type":"float","default":0.0},
            {"key":"gz","question":"gz:","input_type":"float","default":-9.8},
            {"key":"slope","question":"slope:", "input_type":"float","default":0.00},
            
        ]
    },
    {
        "section_name": "Time Series",
        "questions": [
            {"key":"num_time_series_points","question":"num of time series points:","input_type": "int","default":1},
            {"key":"point_num_1","question":"point #1-1:","input_type": "int","default":1},
            {"key":"point_num_1_2","question":"point #1-2:","input_type": "int","default":10},
            {"key":"point_num_1_3","question":"point #1-3:","input_type": "int","default":10},
            {"key":"point_num_1_4","question":"point #1-4:","input_type": "int","default":10}
        ]
    },
]

# Dictionary to store answers
entries = {}

# Function to handle creating a section with its header and questions
def create_section(section, parent_frame):
    # Create section header
    ctk.CTkLabel(parent_frame, text=section["section_name"], font=("Arial", 18, "bold")).pack(pady=10, anchor="w")

    # Create questions within this section
    for question in section["questions"]:
        question_text = question[0]
        input_type = question[1]
        create_question(question_text, input_type, question[2] if len(question) > 2 else None, parent_frame)


# Function to handle creating a question of any type
def create_question(question_text, input_type, options, parent_frame):
    ctk.CTkLabel(parent_frame, text=question_text, anchor="w").pack(pady=5, anchor="w")
    
    # Get the default value from the dictionary, if available
    default_value = default_values.get(question_text, None)
    
    if input_type == "dropdown":
        var = ctk.StringVar(value=default_value if default_value else options[0])
        dropdown = ctk.CTkOptionMenu(parent_frame, variable=var, values=options)
        dropdown.pack(pady=5, anchor="w")
        entries[question_text] = var

    elif input_type == "radio":
        var = ctk.StringVar(value=default_value if default_value else options[0])
        radio_button_1 = ctk.CTkRadioButton(parent_frame, text=options[0], variable=var, value=options[0])
        radio_button_2 = ctk.CTkRadioButton(parent_frame, text=options[1], variable=var, value=options[1])
        radio_button_1.pack(pady=5, anchor="w")
        radio_button_2.pack(pady=5, anchor="w")
        entries[question_text] = var

    elif input_type == "float" or input_type == "int":
        entry = ctk.CTkEntry(parent_frame)
        entry.insert(0, default_value if default_value else "")
        entry.pack(pady=5, anchor="w")
        entries[question_text] = entry


# Iterate through the sections and create them
for section in sections:
    create_section(section, scrollable_frame)

# Button to show advanced options
advanced_button = ctk.CTkButton(scrollable_frame, text="Show Advanced Options", command=lambda: open_advanced_popup(advanced_sections))
advanced_button.pack(pady=20,anchor="w")


# Function to process inputs when the user clicks Submit
def submit_inputs():
    print("Processing inputs...")

    for question, entry in entries.items():
        if isinstance(entry, ctk.StringVar):
            print(f"{question}: {entry.get()}")
        else:
            print(f"{question}: {entry.get()}")

    print("Finished processing inputs.")


# Add a Submit button at the bottom
submit_button = ctk.CTkButton(scrollable_frame, text="Submit", command=submit_inputs)
submit_button.pack(pady=10)

# Start the main loop
root.mainloop()

'''
add if logic
add advanced options - set to defaults if not
add logic so inputs are all required
all logic so floats and ints meet requirements
'''

KeyError: 0

In [ ]:
# Function to process inputs when the user clicks Submit
def submit_inputs():
    print("Processing inputs...")

    for question, entry in entries.items():
        if isinstance(entry, ctk.StringVar):
            print(f"{question}: {entry.get()}")
        else:
            print(f"{question}: {entry.get()}")

    print("Finished processing inputs.")


# Add a Submit button at the bottom
submit_button = ctk.CTkButton(scrollable_frame, text="Submit", command=submit_inputs)
submit_button.pack(pady=10)

In [ ]:
default_values = {
    #numeric parameter defaults
    "pr:" : 0.72,
    "turb_schmidt:": 0.6,
    "beta:": 0.0034,
    "gx:": "0.0",
    "gy:": "0.0",
    "gz:": -9.81,
    "convection scheme:": 3,
    "diffusion scheme:": 3,
    "solver:": 2,
    "multigrid step (ngrid):": 1,
    "multigrid iteration scheme:": 2,
    "multigrid maximum iteration per timestep:": 30,
    "restriction iter:": 2,
    "prolongation iter:": 1,
    "variable or fixed:": "fixed",
    "sweeps:": 25,
    "safety factor:": 0.2,
    "niter:": 20,
    "eps:": 1.0E-5,
    "nswp(1):": 5,
    "nswp(2):": 5,
    "nswp(3):": 5,
    "nswp(4):": 20,
    #flow boundary defaults
    "Friction coefficient:": 'manning',
    "What is the value of the friction coefficient?:": 0.03,
    "Save inflow data (precursor sim.):": "False",
    "Number of inlets:": 5000,
    #synthetic eddy method defaults
    "Velocity Profile:": "1/7th PL",
    "Turbulence intensity:": 0.1,
    "Number inlet profiles:": 1000,
    #modelling options defaults
    "time_averaging": "True",
    #TODO: Define this elsewhere, such as within df creation
    #"t_start_averaging_2": t_start_averaging_1 + 1,
    "noise:": "0.0",
    "LMR:": "new ghost cell approach",
    "normal_ghost_velocity_interpolation:": 2,
    "lrough:": "False",
    "OpenMP threads:": 1,
    "pl_ex (# of extra ghost planes. pl_ex=0 -> blocks have only 1 ghost layer):": 2,
    "Th:": 293,
    "Tc:": 293,
    #energy boundary conditions defaults
    "West Temp Boundary Conditions:": 7,
    "East Temp Boundary Conditions:": 7,
    "South Temp Boundary Conditions:": 7,
    "North Temp Boundary Conditions:": 7,
    "Bottom Temp Boundary Conditions:": 8,
    "Top Temp Boundary Conditions:": 7,
    #free surface parameters defaults
    "L_LSM:": "False",
    "reinit:": "False",
    "ntime_reinit:": 3,
    "reldif_LSM:": 0.001,
    "length:": 0.02,
    "accuracy:": "HJ_WENO",
    #TODO: ask about this value
    "missing value:": 0.65,
    "LENDS:": "False",
    "L_LSMbase:": "False",
    "L_LSMinit:": "False",
    "L_anim_phi:": "False",
     "L_anim_grd:": "False",
    "densl:": 1000.0,
    "densg:": 1.205,
    "nul:": 1.0E-6,
    "nug:": 1.5E-5,
    "gx:": "0.0",
    "gy:": "0.0",
    "gz:": -9.8,
    "slope:": "0.0",
    #time series defaults
    "num of time series points:": 1,
    "point #1-1:": 1,
    "point #1-2:": 10,
    "point #1-3:": 10,
    "point #1-4:": 10
    
}